In [1]:
import requests
import folium
import pandas as pd

# List of relation IDs
relation_ids = [1439251, 1439252, 1439253, 1439254, 1439255]

# Overpass API URL
overpass_url = "https://overpass-api.de/api/interpreter"

# Initialize a map
m = folium.Map(location=[0, 0], zoom_start=2)  # Placeholder coordinates; will update later

# List to store coordinates of each polyline
polyline_coordinates = []

# Loop through each relation ID
for relation_id in relation_ids:
    overpass_query = f"""
    [out:json];
    relation({relation_id});
    out body;
    >;
    out skel qt;
    """

    # Fetch data from Overpass API
    response = requests.post(overpass_url, data={"data": overpass_query})
    if response.status_code != 200:
        print(f"Error fetching data for relation {relation_id}: {response.status_code}")
        continue

    # Parse the JSON response
    data = response.json()

    # Extract nodes and their coordinates
    nodes = {elem["id"]: (elem["lat"], elem["lon"]) for elem in data["elements"] if elem["type"] == "node"}

    # Extract ways (lines connecting nodes)
    ways = [elem["nodes"] for elem in data["elements"] if elem["type"] == "way"]

    # If nodes are found, update the map
    if nodes:
        # Center the map at the first node of the first relation
        if len(m.location) == 2 and m.location == [0, 0]:
            first_node = list(nodes.values())[0]
            m.location = first_node
            m.zoom_start = 6

        # Plot ways on the map and store coordinates
        for way in ways:
            coordinates = [(nodes[node_id][0], nodes[node_id][1]) for node_id in way if node_id in nodes]
            folium.PolyLine(coordinates, color="orange", weight=2.5).add_to(m)

            # Store coordinates in the list
            polyline_coordinates.append({
                "relation_id": relation_id,
                "coordinates": coordinates
            })
    else:
        print(f"No nodes found for relation {relation_id}.")

# Save the map as HTML
m.save("osm_relation_map.html")
print("Map has been saved as 'osm_relation_map.html'")

# Convert the list of polyline coordinates to a DataFrame
df = pd.DataFrame(polyline_coordinates)

# Save DataFrame to CSV
df.to_csv("osm_relation_coordinates.csv", index=False)
print("DataFrame has been saved as 'osm_relation_coordinates.csv'")

Map has been saved as 'osm_relation_map.html'
DataFrame has been saved as 'osm_relation_coordinates.csv'
